### Train model


In [ ]:
import torch
from torch.utils.data import DataLoader 
import numpy as np
import matplotlib.pyplot as plt
import os
from torchmetrics import FBetaScore, Accuracy
import torchvision.transforms as T
from torchvision.transforms import functional as F

import src.dfmaker as dfmaker
import src.BWDatasets as BWDatasets
import src.utility as utility

from paddleocr import PaddleOCR
import paddle


c:\Users\legom\anaconda3\envs\birdword\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\legom\anaconda3\envs\birdword\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [1]:
import sys, site, importlib.util, os
print("PYTHON EXE:", sys.executable)
print("USER_SITE:", site.getusersitepackages() if hasattr(site,'getusersitepackages') else None)
print("SITE-PACKAGES:", site.getsitepackages())
for name in ("paddleocr","ppocr"):
    spec = importlib.util.find_spec(name)
    print(f"{name} ->", spec and (spec.origin or spec))
print("\nsys.path[0..5]:")
print("\n".join(sys.path[:6]))

PYTHON EXE: c:\Users\legom\anaconda3\envs\birdword\python.exe
USER_SITE: C:\Users\legom\AppData\Roaming\Python\Python313\site-packages
SITE-PACKAGES: ['c:\\Users\\legom\\anaconda3\\envs\\birdword', 'c:\\Users\\legom\\anaconda3\\envs\\birdword\\Lib\\site-packages']
paddleocr -> C:\Users\legom\PaddleOCR\paddleocr\__init__.py
ppocr -> None

sys.path[0..5]:
c:\Users\legom\anaconda3\envs\birdword\python313.zip
c:\Users\legom\anaconda3\envs\birdword\DLLs
c:\Users\legom\anaconda3\envs\birdword\Lib
c:\Users\legom\anaconda3\envs\birdword

c:\Users\legom\anaconda3\envs\birdword\Lib\site-packages


In [3]:
seed = 115
generator = torch.Generator()
generator.manual_seed(seed)
np_generator = np.random.default_rng(seed=seed)

In [4]:
device = torch.device('cpu')

# Make sure the notebook is deterministic if training on gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False

print(f'Cuda availability is: {torch.cuda.is_available()}')  # Returns True if a GPU is available
print(f"Training on device {device}.")

Cuda availability is: True
Training on device cuda.


In [ ]:
# Lyngøy
label_path_lyng = "dataset/datasets/lyngoy/ringcodes.csv"
image_path_lyng = "dataset/datasets/lyngoy/images"
bb_path_lyng = "dataset/datasets/lyngoy/labels"

# RF
label_path_rf = "dataset/datasets/rf/ringcodes.csv"
image_path_rf = "dataset/datasets/rf/images"
bb_path_rf = "dataset/datasets/rf/labels"

# Ringmerkingno
label_path_rno = "dataset/datasets/ringmerkingno/ringcodes.csv"
image_path_rno = "dataset/datasets/ringmerkingno/images"
bb_path_rno = "dataset/datasets/ringmerkingno/labels"

max_n_single = 10
max_n_all = None
height = 240 # 240 worked best for the provided dataset

df_lyng = dfmaker.make_dataframe(labels_path=label_path_lyng, img_path=image_path_lyng, bb_path=bb_path_lyng, max_n=max_n_single)
df_rf = dfmaker.make_dataframe(labels_path=label_path_rf, img_path=image_path_rf, bb_path=bb_path_rf, max_n=max_n_single)
df_rno = dfmaker.make_dataframe(labels_path=label_path_rno, img_path=image_path_rno, bb_path=bb_path_rno, max_n=max_n_single)
complete_df = dfmaker.combine_dfs([df_lyng, df_rf, df_rno], shuffle=True)

# Shuffle the dataset to remove order of elements
df_shuffled = complete_df.sample(frac=1).reset_index(drop=True)

# Calculate the splits
train_factor = 0.7
val_factor = 0.15
test_factor = 0.15

train_n = int(train_factor*len(df_shuffled))
val_n = int(val_factor*len(df_shuffled))
test_n = int(test_factor*len(df_shuffled))

# Split data into three dataframes
df_train = df_shuffled.iloc[:train_n]
df_val = df_shuffled.iloc[train_n:train_n+val_n]
df_test = df_shuffled.iloc[train_n+val_n:]

In [14]:
transform = T.Compose([
    T.Lambda(lambda x: F.rotate(x, -90, expand=True)),
    T.Resize((height, utility.find_width(height))) 
])

train_ds = BWDatasets.TrainDataSet(df=df_train, transform=transform, max_n=max_n_all)
val_ds = BWDatasets.TrainDataSet(df=df_val, transform=transform, max_n=max_n_all)
test_ds = BWDatasets.TrainDataSet(df=df_test, transform=transform, max_n=max_n_all)


### Load model

In [ ]:
cfg = "config.json"
model = ...

ModuleNotFoundError: No module named 'ppocr'